In [1]:
import csv
import urllib
import pandas as pd
import numpy as np
import sys
import datetime
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
from progressbar import ProgressBar
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
import fix_yahoo_finance as yf



In [2]:
yf.pdr_override()

In [3]:
#Download lists of all companies in All US exchanges.  Catch Error if urls become outdated.  
NASDAQ_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download'
NYSE_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download'
AMEX_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=AMEX&render=download'
try:
    response = urllib.request.urlopen(NASDAQ_url)
    NASDAQ_df = pd.read_csv(response)
    
    response = urllib.request.urlopen(NYSE_url)
    NYSE_df = pd.read_csv(response)
    
    response = urllib.request.urlopen(AMEX_url)
    AMEX_df = pd.read_csv(response)
    
    df=NASDAQ_df
    df=df.append(NYSE_df)
    df=df.append(AMEX_df)
    
    df=df.drop_duplicates('Symbol')
    df=df.set_index('Symbol')
    df=df.sort_values(by=['MarketCap'],ascending=False)
    
    
except Exception:
    sys.exit("Download Failed.  URL out of date.")

In [5]:
end_date=today=datetime.datetime.now()
start_date= today-datetime.timedelta(days=10000)
fill_history(start_date,end_date,df)

AAPL 954568950980.0 194.21 Technology
[*********************100%***********************]  1 of 1 downloaded
GOOGL 894293777191.5 1285.5 Technology
[*********************100%***********************]  1 of 1 downloaded
GOOG 882348989829.09 1268.33 Technology
[*********************100%***********************]  1 of 1 downloaded
AMZN 877290242432.0 1808.0 Consumer Services
[*********************100%***********************]  1 of 1 downloaded
MSFT 841552740000.0 109.62 Technology
[*********************100%***********************]  1 of 1 downloaded
FB 510207086862.48 176.26 Technology
[*********************100%***********************]  1 of 1 downloaded
BABA 498233728840.68 194.18 Miscellaneous
[*********************100%***********************]  1 of 1 downloaded
JPM 385999365000.0 114.85 Finance
[*********************100%***********************]  1 of 1 downloaded
XOM 356658212972.88 84.24 Energy
[*********************100%***********************]  1 of 1 downloaded
JNJ 349430497309.92 130.

[*********************100%***********************]  1 of 1 downloaded
GILD 101628386629.12 78.16 Health Care
[*********************100%***********************]  1 of 1 downloaded
BKNG 100354714840.45 2083.13 Miscellaneous
[*********************100%***********************]  1 of 1 downloaded
GSK 100090118279.8 40.36 Health Care
[*********************100%***********************]  1 of 1 downloaded
COST 97246835440.0 221.72 Consumer Services
[*********************100%***********************]  1 of 1 downloaded
AZN 96646450839.4 38.15 Health Care
[*********************100%***********************]  1 of 1 downloaded
AVGO 96342538798.4 223.18 Technology
[*********************100%***********************]  1 of 1 downloaded
HDB 96249713820.56 109.24 Finance
[*********************100%***********************]  1 of 1 downloaded
RIO 96090086845.2 55.09 Basic Industries
[*********************100%***********************]  1 of 1 downloaded
BMY 94672467487.36 57.92 Health Care
[*********************

In [4]:
def fill_history(start_date,end_date,df):

    count=0
    pbar = ProgressBar()

    for i,row in df.iterrows():
        symbol=i
        market_cap=row['MarketCap']
        nasdaq_price=row['LastSale']
        sector=row['Sector']
    
    
        if market_cap > 0 and count <100:
            print(symbol,market_cap,nasdaq_price,sector)
            try:
                df_yahoo=pdr.get_data_yahoo(symbol,start_date,end_date)
                df_yahoo.to_csv('saved_data/' + symbol +'.csv', sep='\t', encoding='utf-8')
            except Exception:
                print(symbol + " Error Downloading")
            count+=1



In [178]:
df_yahoo=pdr.get_data_yahoo('AAPL',start_date,end_date)
df_yahoo.to_csv('saved_data/' + symbol +'.csv', sep='\t', encoding='utf-8')

[*********************100%***********************]  1 of 1 downloaded


In [6]:
param = {
    'q': "TURN", # Stock symbol (ex: "AAPL")
    'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NASD", # Stock exchange symbol on which stock is traded (ex: "NASD")
    'p': "1Y" # Period (Ex: "1Y" = 1 year)
}
# get price data (return pandas dataframe)
df_google = get_price_data(param)


